In [1]:
import os
import numpy as np
# import working directory to check functions
os.chdir('/Users/schiend/Desktop/DOHERTY/R-workspace/cecelia/inst')

# config
import py.config_utils as cfg

%load_ext autoreload
%autoreload 2

In [2]:
base_dir = '/Volumes/Analysis_SSD/Dominik/cecelia/projects/8BR53W/ANALYSIS/'
zero_dir = os.path.join(base_dir, '0/s4XQza/')
im_path = os.path.join(zero_dir, 'ccidImage.ome.zarr')
#im_path = os.path.join(zero_dir, 'ccidCorrected.zarr')
#im_path = os.path.join(zero_dir, 'ccidSlidingWindow.zarr')
version_num = 1
task_dir = os.path.join(base_dir, str(version_num), 's4XQza')

In [3]:
from py.napari_utils import NapariUtils

napari_utils = NapariUtils()
napari_utils.viewer = None
napari_utils.open_viewer()
napari_utils.task_dir = task_dir

napari.manifest -> 'skan' could not be imported: Could not find file 'napari.yaml' in module 'skan'


In [4]:
channel_names = [
    'EB', 'T-A', 'T-B', 'AF', 'gen'
]

In [5]:
napari_utils.open_image(
    im_path,
    use_channel_axis = True, as_dask = True,
    downsample_z = False
)

[60, 3, 27, 512, 512]
[512, 512, 27, 3, 60]
['X', 'Y', 'Z', 'C', 'T']
[60, 3, 27, 512, 512]


In [18]:
for j, y in enumerate(napari_utils.dim_utils.spatial_axis()):
    print(y)
    print(j)
    print(napari_utils.dim_utils.dim_idx(y))

X
0
4
Y
1
3
Z
2
2


In [ ]:
  drift_im_shape, dim_utils, shifts_sum
)

In [10]:
napari_utils.im_data[0].shape

(60, 3, 27, 512, 512)

In [7]:
napari_utils.dim_utils.im_dim_order

['T', 'C', 'Z', 'Y', 'X']

In [9]:
napari_utils.dim_utils.dim_vals()

[60, 3, 27, 512, 512]

In [6]:
napari_utils.dim_utils.spatial_axis()

['X', 'Y', 'Z']

In [44]:
# show populations
napari_utils.show_pop_mapping('live')

(live) default/tracked


In [43]:
napari_utils.show_labels_all(
    show_labels = True,
    show_points = False,
    show_tracks = True,
    value_names = ['default']
)

{'base': '/Volumes/Analysis_SSD/Dominik/cecelia/projects/8BR53W/ANALYSIS/1/Bf68An/labels/default.zarr'}
Rendering frames...


100%|███████████████████████████████████████████| 39/39 [00:05<00:00,  7.02it/s]


In [ ]:
# save image as tiff
import py.tiff_utils as tiff_utils

tiff_utils.save_as_tiff(
    '/Volumes/USER_data/Dominik/Experiments/TEAGEN_SKIN/DATA/PuJo5p.tif',
    napari_utils.im_data[0], napari_utils.dim_utils, imagej = True)

In [ ]:
print('here')

In [ ]:
napari_utils.im_scale

In [42]:
from napari_animation import AnimationWidget
animation_widget = AnimationWidget(napari_utils.viewer)
napari_utils.viewer.window.add_dock_widget(animation_widget, area='right')

In [ ]:
pop_type = 'live'
pop_map = napari_utils.pop_utils.pop_map(napari_utils.task_dir, pop_type)
pop_data = napari_utils.pop_utils.pop_data(napari_utils.task_dir, pop_type)

In [ ]:
pop_info = pop_map['5iaWSo']

In [ ]:
pop_info